In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import nltk
import string
import matplotlib.pyplot as plt
import random
%matplotlib inline
import os
import json

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import decomposition, ensemble
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

import joblib

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os,boto3,sys,glob,json

### Part 1  Loading the  data 

#### Read in csv file

In [2]:
testDF=pd.read_csv('run1000charts_0521.csv')
# the csv is generated in D drive D:/chartai_qa  with extract1000charts_jsontxt.py, then uploaded to here

In [3]:
len(testDF)

76792

In [4]:
testDF.columns

Index(['Unnamed: 0', 'chart_id', 'code', 'text', 'start_offset', 'end_offset',
       'text75'],
      dtype='object')

In [5]:
testDF=testDF.drop(['Unnamed: 0'],axis=1)

In [6]:
testDF.head()

,chart_id,code,text,start_offset,end_offset,text75
0,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E063,Ht,1187,1189,ocedure codes 36415 venipunct routine vital si...
1,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,1374,1388,tion list reviewed reconciled patient past med...
2,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,1391,1403,reconciled patient past medical history hyperl...
3,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,2957,2971,tion list reviewed reconciled patient past med...
4,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,2974,2986,reconciled patient past medical history hyperl...


In [7]:
# just check if there's null text, if yes, remove them 
nulltext=testDF[~pd.notnull(testDF['text'])] 
len(nulltext)

0

In [8]:
nulltext=testDF[~pd.notnull(testDF['text75'])] 
len(nulltext)

26

In [9]:
nulltext   # there are some text not correctly extracted

,chart_id,code,text,start_offset,end_offset,text75
1336,IBXRMA2018_MULT_1164353980010001_HMK_160303709...,G129,SMA,4,7,NaN
2198,IBXRMA2018_MULT_1176161080010001_HMK_192365603...,N08,Kidney,46,52,NaN
3876,IBXRMA2018_MULT_1202740400010001_HMK_153425750...,G129,SMA,4,7,NaN
4032,IBXRMA2018_MULT_1202908350010001_HMK_210648786...,E744,PC,25,27,NaN
5591,IBXRMA2018_MULT_1210779380010001_HMK_184462858...,G129,SMA,4,7,NaN
6465,IBXRMA2018_MULT_1212745310010001_HMK_165441851...,N08,Kidney,48,54,NaN
9116,IBXRMA2018_MULT_1216729130010001_HMK_196286386...,N08,Kidney,46,52,NaN
14970,IBXRMA2018_MULT_1216856710010001_HMK_173309591...,G129,SMA,4,7,NaN
18950,IBXRMA2018_MULT_1216874020010001_HMK_143364782...,E744,PC,25,27,NaN
21748,IBXRMA2018_MULT_1216901800010001_HMK_201224196...,G129,SMA,4,7,NaN


In [10]:
testDF=testDF[pd.notnull(testDF['text75'])] 
len(testDF)

76766

In [11]:
testDF['chart_id'].nunique()   # only 980 charts not 1044 charts

980

In [12]:
testDF=testDF.reset_index(drop=True)   # very very import, to reset the index, otherwise the below running the model will not be right, because I use the index as reference for position and matching them
testDF.tail()

,chart_id,code,text,start_offset,end_offset,text75
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...


In [13]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle

Using TensorFlow backend.


In [14]:
testDF.columns

Index(['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75'], dtype='object')

### Part 2. Load one LR consolidaetd model  and apply to test data

In [92]:
# load the lstm model
model2=joblib.load('../../picklefolder_ngrams/refreshed_models_update0531/consolidated_LRMay2919.pickle')


In [213]:
predictiont2 = model2.predict_proba(testDF['text75'])
#prediction = np.where(pred[:,1] > 0.5, 1, 0)
finallabelt1= (predictiont2[:,1] > 0.5).astype(np.int) 
finallabelt2= (predictiont2[:,1] > 0.3).astype(np.int) 
finallabelt3= (predictiont2[:,1] > 0.2).astype(np.int)

In [214]:
testDF['pred']=finallabelt3

In [215]:
testDF=testDF.reset_index(drop=True)
testDF.tail()

,chart_id,code,text,start_offset,end_offset,text75,pred
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,1
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,1
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,1
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,0
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,1


In [216]:
print(len(set(testDF['chart_id'].tolist())))


980


In [217]:
# to make minimal change in the following code, 
df_pred=testDF

###  Part 3. Load in coder feedback and  evaluate the accuracy

#### Select common chart_ids from both 1000 model run and 6000 coder annotation

In [218]:
# model computation
df_pred1=df_pred[['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75','pred']]
print(len(df_pred1))
df_pred1 = df_pred1.drop_duplicates()
print(len(df_pred1))

76766
76766


In [219]:
# read in all charts with coder feedback
df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_with_chartids_29042019.csv')
print(len(df_codeall))

136527


In [220]:
print("unique chart is in 1000 charts run model output:",df_pred1['chart_id'].nunique())
print("unique chart is in 6000 charts with coder feedback:",df_codeall['chart_id'].nunique())

unique chart is in 1000 charts run model output: 980
unique chart is in 6000 charts with coder feedback: 5982


In [221]:
# now find overlap chart ids
overlapped_charts = pd.merge(df_codeall[['chart_id']].drop_duplicates(),df_pred1[['chart_id']].drop_duplicates(), how = 'inner')
overlapped_charts = list(overlapped_charts['chart_id'].unique())
print(len(overlapped_charts))  

798


In [222]:
# select the model output
fd_prod = df_pred1[['chart_id','code','start_offset','end_offset','text75','pred']][df_pred1['chart_id'].isin(overlapped_charts)].drop_duplicates()
len(fd_prod)

60131

In [223]:
fd_prod['pred'].value_counts()  # 1 delete

1    37019
0    23112
Name: pred, dtype: int64

In [224]:
# select those in the code output
data = df_codeall[df_codeall['chart_id'].isin(overlapped_charts)]
print(len(data))
print(len(data[data['label'] == 'added']))
print(len(data[data['label'] == 'deleted']))
print(len(data[data['label'] == 'agreed']))

30057
17974
3232
8851


#### The accurary number for the pipeline models

In [225]:
# Use fuzzy logic to compute the deleted, added, agreed
data = data[data['start'] != 0]
data = data[data['end'] != 0]
#data = data[data['created_at'] == date ] ##Change date to filter for datewise data
#label_counts = data.groupby(['label']).size().reset_index(name='counts')
instmatches = data[data['label'] == 'agreed']
instfuzzymatches = data[data['label'] == 'deleted']
instfuzzymatches = instfuzzymatches[instfuzzymatches['deleted_reason'].isin(['Incorrect Specification - Non-Risk Adjusted','repeated_instance','incorrect_year_of_service']) ]
instmatches = pd.concat([instmatches, instfuzzymatches])
instmatches = instmatches[['chart_id','submission_id','code_id','start']].drop_duplicates()

instdeleted = data[data['label'] == 'deleted']
instdeleted = instdeleted[instdeleted['deleted_reason'] != 'other']
instdeleted = pd.merge(instdeleted, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start'], indicator = True)
instdeleted = instdeleted[instdeleted['_merge'] == 'left_only']
instdeleted = instdeleted[['chart_id','submission_id','code_id','start','deleted_reason']].drop_duplicates()
deleted_reason = instdeleted.groupby(['deleted_reason']).size().reset_index(name = 'counts')
instdeleted = instdeleted[['chart_id','submission_id','code_id','start']].drop_duplicates()


instadded = data[data['label'] == 'added']
instadded = pd.merge(instadded, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()
instadded = pd.merge(instadded, instdeleted,how = 'outer', on = ['chart_id','submission_id','code_id','start'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)
print(added)
print(deleted)
print(agreed)

16098
770
9168


In [226]:
len(fd_prod)

60131

In [227]:
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']
prod_additions_models = pd.merge(instadded, fd_prod,how = 'inner', on = ['chart_id'], indicator = True)
prod_additions_models['abs_diff_start'] = abs(prod_additions_models['start_offset_x'] - prod_additions_models['start_offset_y'])
prod_additions_models['abs_diff_end'] = abs(prod_additions_models['end_offset_x'] - prod_additions_models['end_offset_y'])
prod_additions_models1 = prod_additions_models[ (
                    (prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_start'] <= 75))
          | (prod_additions_models['start_offset_x'] == prod_additions_models['start_offset_y'])
          | (prod_additions_models['end_offset_x'] == prod_additions_models['end_offset_y'])
          |((prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_end'] <= 75))
          ]
instadded_modeldeleted = prod_additions_models1[['chart_id','submission_id','code','start_offset_y','end_offset_y']].drop_duplicates()

In [228]:
len(instadded_modeldeleted)

15128

In [229]:
# initial numbers considering all the additions data for recall

total = len(instmatches) + len(instdeleted) + len(instadded)
added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))


precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)

TP: 9168
TN: 48048
FN: 16098
FP: 770
precision: 0.9225196216542564
recall: 0.3628591783424365
accuracy: 0.7723125101236434
-------------------
Agreements =  9168
Deletions =  770
Additions =  16098


In [230]:
# initial numbers considering model deleted additions data for recall

total = len(instmatches) + len(instdeleted) + len(instadded_modeldeleted)
added = len(instadded_modeldeleted)
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN1= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)

TP: 9168
TN: 48048
FN: 15128
FP: 770
precision: 0.9225196216542564
recall: 0.377346065195917
accuracy: 0.7825587438794211
-------------------
Agreements =  9168
Deletions =  770
Additions =  15128


####  Refreshed  Model accuracy numbers

In [231]:
print(len(df_pred1))   # model output with all the instances

76766


In [232]:
print(len(fd_prod))  # already selected  the 798 chart  from previous steps

60131


In [233]:
fd_prod.dtypes

chart_id        object
code            object
start_offset     int64
end_offset       int64
text75          object
pred             int64
dtype: object

In [234]:
TN=len(fd_prod[fd_prod['pred']==1])  # deleted instances but it's not TN true negative
print("TN:",TN)

TN: 37019


In [235]:
# select those 708 charts and only passed the filtering not deleted, 0 is keep here
dev_df_sub_1 = fd_prod[fd_prod['pred']==0]  # select the 798 chart
len(dev_df_sub_1[['chart_id','start_offset']].drop_duplicates())  # 36701 pass through

23112

In [236]:
dev_df_sub_1.rename(columns={'start_offset':'start','end_offset':'end'},inplace=True)

In [237]:
dev_df_sub_1.rename(columns={'code':'code_id'},inplace=True)

In [238]:
new_matches = pd.merge(dev_df_sub_1[['chart_id','start']], instmatches,how = 'inner', on = ['chart_id','start'], indicator = True)
new_deleted = pd.merge(dev_df_sub_1[['chart_id','start']], instdeleted,how = 'inner', on = ['chart_id','start'], indicator = True)

In [239]:
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
new_added = pd.merge(dev_df_sub_1, instadded,how = 'inner', on = ['chart_id'], indicator = True)
new_added['abs_diff_start'] = abs(new_added['start_x'] - new_added['start_y'])
new_added['abs_diff_end'] = abs(new_added['end_x'] - new_added['end_y'])

In [240]:
new_added1 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 5))
          | (new_added['start_x'] == new_added['start_y'])
          ]

In [241]:
#1. if the start offset matches exactly
#2. If the end offset matches exactly
#3. If the overlap of start offset is between 75 characters and the code id matches
#4. If the overlap of end offset is between 75 characters and the code id matches
new_added2 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 75))
          | (new_added['start_x'] == new_added['start_y'])
          | (new_added['end_x'] == new_added['end_y'])
          |((new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_end'] <= 75))
          ]

In [242]:
instadded_modeldeleted.columns

Index(['chart_id', 'submission_id', 'code', 'start_offset_y', 'end_offset_y'], dtype='object')

In [243]:
#just validating the additions with a different method of calculation
instadded_modeldeleted.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
x = pd.merge(dev_df_sub_1[['chart_id','start']].drop_duplicates()
             , instadded_modeldeleted[['chart_id','start']].drop_duplicates()
             ,how = 'inner', on = ['chart_id','start'], indicator = True)
len(x)

9350

In [244]:
print(len(new_matches))
print(len(new_deleted))
print(len(new_added1[['chart_id','start_x']].drop_duplicates()))
print(len(new_added2[['chart_id','start_x']].drop_duplicates()))

7578
497
6679
9350


In [245]:
#type 1 - older number of additions considering only start offset
refreshed_agreements1 = int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions1 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions1 = added - int(len(new_added1[['chart_id','start_x']].drop_duplicates()))

#type 2 - new number of additions considering start offset and end offset
refreshed_agreements2 = int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions2 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions2 = added - int(len(new_added2[['chart_id','start_x']].drop_duplicates()))

print(refreshed_agreements1)
print(refreshed_deletions1)
print(refreshed_additions1)
print("--------")
print(refreshed_agreements2)
print(refreshed_deletions2)
print(refreshed_additions2)

14257
8855
8449
--------
16928
6184
5778


In [246]:
refreshed_total1 = refreshed_agreements1 + refreshed_deletions1 + refreshed_additions1
refreshed_total2 = refreshed_agreements2 + refreshed_deletions2 + refreshed_additions2
print(refreshed_total1)
print(refreshed_total2)

31561
28890


In [247]:
#type 1 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements1*100/refreshed_total1,'%')
print('Deletions = ', refreshed_deletions1*100/refreshed_total1,'%')
print('Additions = ', refreshed_additions1*100/refreshed_total1,'%')
print('Precision = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_deletions1),'%')
print('Recall = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_additions1),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements1)
print('Deletions = ', refreshed_deletions1)
print('Additions = ', refreshed_additions1)

TP = refreshed_agreements1
FP = refreshed_deletions1
FN = refreshed_additions1

TN=len(fd_prod[fd_prod['pred']==1])-FN # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)


Instance level New Models:
Agreements =  45.17283989734165 %
Deletions =  28.056778936028643 %
Additions =  26.7703811666297 %
Precision =  61.686569747317414 %
Recall =  62.78957103849203 %
-------------------
Agreements =  14257
Deletions =  8855
Additions =  8449
TP: 14257
TN: 28570
FN: 8449
FP: 8855
precision: 0.6168656974731741
recall: 0.6278957103849203
accuracy: 0.7122283015416341


In [248]:
#type 2 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements2*100/refreshed_total2,'%')
print('Deletions = ', refreshed_deletions2*100/refreshed_total2,'%')
print('Additions = ', refreshed_additions2*100/refreshed_total2,'%')
print('Precision = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_deletions2),'%')
print('Recall = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_additions2),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements2)
print('Deletions = ', refreshed_deletions2)
print('Additions = ', refreshed_additions2)



TP = refreshed_agreements2
FP = refreshed_deletions2
FN = refreshed_additions2

TN=len(fd_prod[fd_prod['pred']==1])-FN  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)



Instance level New Models:
Agreements =  58.594669435790934 %
Deletions =  21.40533056420907 %
Additions =  20.0 %
Precision =  73.24333679473867 %
Recall =  74.55298159076897 %
-------------------
Agreements =  16928
Deletions =  6184
Additions =  5778
TP: 16928
TN: 31241
FN: 5778
FP: 6184
precision: 0.7324333679473867
recall: 0.7455298159076896
accuracy: 0.8010676689228517
